In [4]:
import json
import matplotlib.pyplot as plt
import numpy as np
import logging
import faulthandler
from concurrent.futures import ThreadPoolExecutor, as_completed
from sdrfly.sdr.sdr_simulated import SimulatedBluetoothSDR
from sdrfly.sdr.sdr_hackrf import HackRFSdr
from sdrfly.channelizers.channelizer_fft import ChannelizerFFT
from sdrfly.channelizers.channelizer_numba import ChannelizerNumba
from sdrfly.demodulators.demodulator_numba import GFSKDemodNumba
from sdrfly.utils import detect_peaks, extract_lap_and_access_code, probe_devices

# Enable faulthandler
faulthandler.enable()

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s.%(msecs)03d %(levelname)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

# Suppress Numba debug output
import numba
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

# Load configuration from JSON file
config_path = '/home/dev/workspace/bluetooth-demod/src/sdrfly/config.json'
with open(config_path, 'r') as f:
    config = json.load(f)

START_FREQ = config["START_FREQ"]
END_FREQ = config["END_FREQ"]
STEP_FREQ = config["STEP_FREQ"]
SAMPLE_RATE = config["SAMPLE_RATE"]
BANDWIDTH = config["BANDWIDTH"]
NUM_SAMPLES = config["NUM_SAMPLES"]
PEAK_THRESHOLD = config["PEAK_THRESHOLD"]
GAIN = config["GAIN"]
NUM_CHANNELS = config["NUM_CHANNELS"]
NUM_THREADS = config["NUM_THREADS"]
TARGET_LAP = config["TARGET_LAP"]

def process_channel(channel_idx, ch_samples):
    logger.info(f"Processing channel {channel_idx + 1}")

    # Initialize GFSK demodulator
    gfsk_demod = GFSKDemodNumba(0.5)  # Adjust kf value as needed

    # Demodulate the signal using Numba
    demodulated_signal = gfsk_demod.demodulate(ch_samples)

    # Detect peaks in the demodulated signal
    peaks = detect_peaks(demodulated_signal, PEAK_THRESHOLD)

    if len(peaks) == 0:
        logger.debug(f"No peaks detected in channel {channel_idx + 1}.")
        return None

    logger.info(f"Detected peaks at indices: {peaks} in channel {channel_idx + 1}")
    lap_and_access_codes = extract_lap_and_access_code(demodulated_signal, peaks)
    
    for idx, (access_code, lap) in enumerate(lap_and_access_codes):
        logger.info(f"Channel {channel_idx + 1}, Peak {idx}: Access Code: {access_code}, LAP: {lap}")
        if lap == TARGET_LAP:
            logger.info(f"Found target LAP {TARGET_LAP} in channel {channel_idx + 1}!")
            return demodulated_signal, peaks

    return None

def main():
    # Probe for available devices
    probe_devices()

    # Initialize SDR (choose the appropriate SDR class)
    sdr = HackRFSdr(START_FREQ, SAMPLE_RATE, BANDWIDTH, GAIN)
    # sdr = SimulatedBluetoothSDR(START_FREQ, SAMPLE_RATE, BANDWIDTH, GAIN)

    # Initialize Channelizer
    channelizer = ChannelizerFFT(num_channels=NUM_CHANNELS, channel_bw=BANDWIDTH, sample_rate=SAMPLE_RATE)
    # You can switch between different channelizer methods by initializing with ChannelizerNumba
    
    found = False

    current_freq = START_FREQ

    while not found and current_freq <= END_FREQ:
        logger.info(f"Setting frequency to {current_freq / 1e6} MHz")
        sdr.set_frequency(current_freq)

        logger.info("Capturing wideband samples...")
        samples = sdr.capture_samples(NUM_SAMPLES)

        # Check if samples are captured correctly
        if not np.any(samples):
            logger.error("No samples captured. Please check the SDR connection and settings.")
            current_freq += STEP_FREQ
            continue

        # Channelize the wideband samples into multiple 1 MHz channels
        channel_samples = channelizer.channelize(samples)

        with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
            future_to_channel = {executor.submit(process_channel, idx, ch_samples): idx for idx, ch_samples in enumerate(channel_samples)}

            for future in as_completed(future_to_channel):
                result = future.result()
                if result is not None:
                    demodulated_signal, peaks = result
                    found = True
                    break

        if not found:
            current_freq += STEP_FREQ

    if found:
        # Plot the demodulated signal of the last checked channel
        plt.figure(figsize=(12, 6))
        plt.plot(demodulated_signal)
        plt.plot(peaks, demodulated_signal[peaks], "x")
        plt.title('Demodulated Bluetooth Signal with Detected Peaks')
        plt.xlabel('Sample')
        plt.ylabel('Amplitude')
        plt.grid(True)
        plt.show()

    sdr.close()

# Run the main function for testing
if __name__ == '__main__':
    main()

[INFO] Opening HackRF One #0 88869dc2a59211b...
2024-06-29 11:03:14.305 INFO Setting frequency to 2407.0 MHz
2024-06-29 11:03:14.307 INFO Capturing wideband samples...


Found device: {device=HackRF One, driver=hackrf, label=HackRF One #0 88869dc2a59211b, part_id=a000cb3c005d435b, serial=0000000000000000088869dc2a59211b, version=2021.03.1}


2024-06-29 11:03:14.915 ERROR No samples captured. Please check the SDR connection and settings.
2024-06-29 11:03:14.918 INFO Setting frequency to 2408.0 MHz
2024-06-29 11:03:14.923 INFO Capturing wideband samples...
2024-06-29 11:03:15.536 ERROR No samples captured. Please check the SDR connection and settings.
2024-06-29 11:03:15.540 INFO Setting frequency to 2409.0 MHz
2024-06-29 11:03:15.544 INFO Capturing wideband samples...
2024-06-29 11:03:16.157 ERROR No samples captured. Please check the SDR connection and settings.
2024-06-29 11:03:16.161 INFO Setting frequency to 2410.0 MHz
2024-06-29 11:03:16.165 INFO Capturing wideband samples...
2024-06-29 11:03:16.776 ERROR No samples captured. Please check the SDR connection and settings.
2024-06-29 11:03:16.777 INFO Setting frequency to 2411.0 MHz
2024-06-29 11:03:16.781 INFO Capturing wideband samples...
2024-06-29 11:03:17.721 INFO Processing channel 1
2024-06-29 11:03:20.066 DEBUG No peaks detected in channel 1.
2024-06-29 11:03:20.